In [1]:
import os, re, sys
import numpy as np
import pickle as pkl

In [2]:
input_folder = "../data/preprocessed_tika/"


In [3]:
sys.path.append("../train/")

from simulation_classifier_annotations import annotations

In [4]:
file_to_text = {}
for root, folder, files in os.walk(input_folder):
    for file_name in files:
        if not file_name.endswith("pkl"):
            continue
        pdf_file = re.sub(".pkl", "", file_name)
        
        full_file = input_folder + "/" + file_name
#         print (full_file)
        with open(full_file, 'rb') as f:
            text = pkl.load(f)
        file_to_text[pdf_file] = text

In [5]:
sap_regex = re.compile(r"(?i)statistical\s*analysis\s*plan")

sap_regex = re.compile(r"(?i)\bSAP\b")

sap_regex = re.compile(r"(?i)\bdata\s*analysis\s*plan\b")

sap_regex = re.compile(r"(?i)\bDAP\b")


sap_regex = re.compile(r"(?i)\b(simulations?)\b")
sap_regex = re.compile(r"(?i)\b(scenarios?|simulations?|tmle|estimation|simulated?s?|simulating|finite sample|sampled|bayesian|monte carlo|kolmogorov)\b")

In [6]:
file_to_text["TB-TB-2018-M-P-A.pdf"][146]

'\n\t\nStatistical\tanalysis\tplan,\tdated\t14\tDecember\t2016 \n\n'

In [14]:
#  Annotations are page number (zero-indexed)
annotations = {
    'TB-TB-2018-DAR-PIA.pdf': [],
    'TB-TB-2018-M-P-A.pdf': [],
    'HIV-HIV-2016-Ayles-A Study of the HIV prevention Trials Network.pdf': [80],
    'TB-TB-2019-T-A-b.pdf': [],
    'HIV-HIV-2016-Laufer-Trimethoprim-Sulfamethoxazole-Chloroquine.pdf': [],
    'NTD-Filariasis-2019-Weil-DOLF IDA Papua New Guinea.pdf': [],
    'HIV-HIV-2017-Abrams-IMPAACT P1101.pdf': [92, 94],  # Monte Carlo simulations
    'NUT-A-2019-M-T.pdf': [],
    'HIV-HIV-2018-IMPAACT-P1112.pdf': [],  # Uses simulation/Monte Carlo for PK not for sample size
    'EDD-rotavirus-2017-Isanaka-ROSE.pdf': [],
    'NTD-Y-2019-M-Y.pdf': [],
    'NTD-Chagas-2015-Morillo-BENEFIT.pdf': [],
    'MAT-V-2018-R-M.pdf': [], # page 31 (30 zero-indexed) has a graph of detectable z-score vs sample size but this does not look like it is simulation-derived.
    'HIV-HIV-2018-GSK1265744.pdf': [], # Uses simulation for dose but not sample size
    "POL-P-2018-O-A-L.pdf": [],
    'MAT-Hypothermia-2018-Hansen-Electricity-free Infant Warmer.pdf': [], # light   bulbs to simulate an infant’s metabolism
    'EDD-T-2019-S-T.pdf': [],
    'NTD-Hookwork-2018-Keiser-Efficacy and safety of a single.pdf': [],
    'TB-TB-2018-Piazza-A Phase 1b, Randomized, Double-blind.pdf': [],
    'MAL-Malaria-2019-Llanos-Cuentas-Tafenoquine vs. Primaquine to.pdf': [],
    'HIV-HIV-2017-Corey-HVTN703-HPTN081.pdf': [45, 47, 51],
    # These power calculations are conducted using the open source R package seqDesign developed by the protocol statisticians [117], which computes power based on simulating many thousands of efficacy trials, applying  the sequential monitoring procedures to each trial, and computing power as the  fraction of the trials where the 1-sided 0.025-level Wald test rejects the null  hypothesis. The calculations are based on a large number of simulated efficacy  trials.
    'MAL-M-2019-A-S.pdf': [63],
    # least 80% power using 10,000 simulations with the data from Sumba
    'NTD-E-2017-A-A.pdf': [],
    'HIV-HIV-2017-Bekker-HVTN100.pdf': [52, 53],  # omputer simulations (details provided below)
    'MAL-Malaria-2019-Tinto-Extension to study MALARIA-055 PRI.pdf': [],
    'NUT-NUT-2019-A-I.pdf': [],
    # 'HIV-HIV-2018-M-A.pdf': [], # image based PDF - exclude from dataset
    'HIV-HIV-2017-L-I.pdf': [],
    'TB-TB-2019-M-A.pdf': [],
    'VAC-G-2016-VAC-G-2016-V.pdf': [],
    'HIV-HIV-2016-N-S.pdf': [], #  Uses simulation but not for sample size
    'VAC-Shigellosis-2019-Raqib-Shigella WRSS1 Vaccine trial in Bangladesh.pdf': [],
    'HIV-HIV-2017-Evolocumab-AMG145.pdf': [],
    'TB-TB-2015-D-A.pdf': [], # Using the sample size calculation formula
    'POL-P-2016-G-T.pdf': [],
    'TB-TB-2019-Suliman-A Phase I_IIa Double-Blind.pdf': [],
    # 'MAT-Anemia-2015-Etheredge-Prenatal Iron Supplements Safety and.pdf': [],
    # 'HIV-HIV-2016-R-E.pdf': [],
    'HIV-HIV-2019-H-S.pdf': [57, 63, 136, 138, 144, 169, 170, 172, 175, 176, 180, 181, 182, 183, 184, 186, 189, 192, 193, 194, 199, 206, 209, 212, 215, 238], # a lot of this seems to deal with simulation for sample size
    # 'NTD-Filariasis-2019-Weil-DOLF IDA Haiti.pdf': [],
    # 'MAT-MAT-2018-G-F.pdf': [],
    # 'EDD-R-2017-G-V.pdf': [],
    'MAT-HIV-2016-Fowler-PROMISE.pdf': [22, 118, 119, 351, 539, 776], # Not sure - need to revisit
    'TB-TB-2018-H-A.pdf': [54, 137],
    'HIV-HIV-2016-HVTN114.pdf': [33],
    'HIV-HIV-2018-Long-acting Cabotegravir Plus Long-acting Rilpivirine.pdf': [], # Not sure
    'NUT-S-2018-R-S.pdf': [95], # Unsure - see p 95 (zero indexed)
    # 'VAC-Tdap-2019-Sancovski-A Post-marketing, Observational, Retrospective.pdf': [],
    'NTD-t-2017-M-E.pdf': [],
    # 'HIV-HIV-2018-Lockman-Dolutegravir-Containing versus Efavirenz-Containing Antiretroviral Therapy.pdf': [],
    # 'MAL-M-2018-D-C.pdf': [],
    # 'NTD-D-2017-S-P.pdf': [],
    'HIV-HIV-2016-Garrett-HVTN108.pdf': [52], # These simulations are based on 10,000 simulated datasets and use an exchangeable correlation structure between responses with pairwise     correlations of 0.3
    # 'HIV-HIV-2018-Labhardt-CASCADE.pdf': [],
    # 'TB-TB-2019-Dawson-A Phase 2 Open‐Label Partially .pdf': [],
    # 'MAL-MAL-2018-Dorsey-PROMOTE Birth Cohort 1.pdf': [],
    # 'PNE-P-2019-C-P.pdf': [],
    # 'HIV-HIV-2019-Venter-ADVANCE.pdf': [],
    # 'HIV-HIV-2019-K-C.pdf': [],
    'PNE-Pneumonia-2019-Alexander-LEAP2.pdf': [48, 49],  # Monte Carlo simulation were utilized
    # 'MAT-Cervical Disease-2019-Greene-LEEP.pdf': [],
    'TB-TB-2018-Churchyard-Bedaquiline-Delamanid.pdf': [73],
    # 'MAL-Malaria-2019-Foy-RIMDAMAL.pdf': [],
    # 'TB-TB-2019-N-S.pdf': [],
    # 'NUT-Stunting-2019-Humphrey-SHINE.pdf': [],
    'PNE-Pneumonia-2018-Keenan-Mortality Reduction After Oral Azithromycin.pdf': [],
    # Use Monte Carlo but only for analysis, not for sample size planning
    'EDD-Cholera-2016-Qadri-Shanchol.pdf': [], # Kolmogorov used but not for sample size - just for Bayesian p-values.
    # 'TB-TB-2020-John-Stewart-iTIPS.pdf': [],
    # 'VAC-Ecoli-2019-Qadri-ETVAX.pdf': [],
    'HIV-HIV-2018-Havlir-SEARCH.pdf': [65, 66, 73],  # We will also using Monte Carlo multi-variable simulations to estimate
    # 'HIV-HIV-2016-Kurth-High-yield HIV testing, facilitated linkage to care.pdf': []
    'HIV-HIV-2017-Corey-HVTN704-HPTN085.pdf': [42, 43, 48],
    # 'HIV-HIV-2016_Tebas-A5354.pdf': [],
    # 'HIV-HIV-2016-Weinberg-NICHP1091.pdf': [],
    # 'MAL-Malaria-2016-Valencia-Phase 1 and 2a Clinical.pdf': [],
    # 'NTD-Filariasis-2019-Weil-DOLF IDA Indonesia.pdf': [],
    # 'VAC-M-2015-L-W.pdf': []
}


In [15]:
r1 = re.compile(r"(?i)sample\s*size")
r2 = re.compile(r"(?i)simulat")
lastm1 = []
lastm2 = []
lastpage = ""
for file, pages in file_to_text.items():
    print (file, len(pages))
    for page_no, page_text in enumerate(pages):
#         if "sample" not in page_text.lower():
#             continue
        if page_no in annotations.get(file, []):
            continue
        m1 = list(r1.finditer(page_text))
        m2 = list(r2.finditer(page_text))
        if (len(m1) > 0 and len(lastm2) > 0) or (len(lastm1) > 0 and len(m2) > 0):      
            for match in m2:
                l = int(np.max((0, match.start() - 30)))
                r = int(np.min((len(page_text), match.end() + 30)))
                print ("\t", page_no, "\t",  page_text[l:r].replace("\n", " "))
            for match in lastm2:
                l = int(np.max((0, match.start() - 30)))
                r = int(np.min((len(lastpage), match.end() + 30)))
                print ("\t", page_no, "\t",  lastpage[l:r].replace("\n", " "))    
        lastm1 = m1
        lastm2 = m2
        lastpage = page_text

TB-TB-2018-DAR-PIA.pdf 68
TB-TB-2018-M-P-A.pdf 161
HIV-HIV-2016-Ayles-A Study of the HIV prevention Trials Network.pdf 166
TB-TB-2019-T-A-b.pdf 389
	 387 	 lford, Alvan R. Feinstein. A  simulation study of the number of eve
HIV-HIV-2016-Laufer-Trimethoprim-Sulfamethoxazole-Chloroquine.pdf 100
NTD-Filariasis-2019-Weil-DOLF IDA Papua New Guinea.pdf 90
HIV-HIV-2017-Abrams-IMPAACT P1101.pdf 85
NUT-A-2019-M-T.pdf 196
HIV-HIV-2018-IMPAACT-P1112.pdf 155
EDD-rotavirus-2017-Isanaka-ROSE.pdf 210
NTD-Y-2019-M-Y.pdf 66
NTD-Chagas-2015-Morillo-BENEFIT.pdf 76
MAT-V-2018-R-M.pdf 193
HIV-HIV-2018-GSK1265744.pdf 124
	 53 	 l Pharmacology Modelling and  Simulation (CPMS) at GlaxoSmithKline.
POL-P-2018-O-A-L.pdf 97
MAT-Hypothermia-2018-Hansen-Electricity-free Infant Warmer.pdf 56
	 23 	 in an   environmental chamber simulating room temperature (about 22
	 23 	   2. Estimated use: This test simulates conditions of the pad in us
	 23 	 mall (3.8 W) light   bulbs to simulate an infant’s metabolism, and 
	 2

In [17]:
sap_regex = re.compile(r"(?i)\btmle\b")
for file, pages in file_to_text.items():
#     if file in annotations:
#         continue
    print (file, len(pages))
    for page_no, page_text in enumerate(pages):
#         if "sample" not in page_text.lower():
#             continue
        matches = sap_regex.finditer(page_text)
               
        for match in matches:
            l = int(np.max((0, match.start() - 30)))
            r = int(np.min((len(page_text), match.end() + 30)))
            print ("\t", page_no, "\t",  page_text[l:r].replace("\n", " "))

TB-TB-2018-DAR-PIA.pdf 68
TB-TB-2018-M-P-A.pdf 161
HIV-HIV-2016-Ayles-A Study of the HIV prevention Trials Network.pdf 166
TB-TB-2019-T-A-b.pdf 389
HIV-HIV-2016-Laufer-Trimethoprim-Sulfamethoxazole-Chloroquine.pdf 100
NTD-Filariasis-2019-Weil-DOLF IDA Papua New Guinea.pdf 90
HIV-HIV-2017-Abrams-IMPAACT P1101.pdf 85
NUT-A-2019-M-T.pdf 196
HIV-HIV-2018-IMPAACT-P1112.pdf 155
EDD-rotavirus-2017-Isanaka-ROSE.pdf 210
NTD-Y-2019-M-Y.pdf 66
NTD-Chagas-2015-Morillo-BENEFIT.pdf 76
MAT-V-2018-R-M.pdf 193
HIV-HIV-2018-GSK1265744.pdf 124
POL-P-2018-O-A-L.pdf 97
MAT-Hypothermia-2018-Hansen-Electricity-free Infant Warmer.pdf 56
EDD-T-2019-S-T.pdf 105
NTD-Hookwork-2018-Keiser-Efficacy and safety of a single.pdf 51
TB-TB-2018-Piazza-A Phase 1b, Randomized, Double-blind.pdf 56
MAL-Malaria-2019-Llanos-Cuentas-Tafenoquine vs. Primaquine to.pdf 402
HIV-HIV-2017-Corey-HVTN703-HPTN081.pdf 193
	 63 	 aximum likelihood estimation (tMLE) will be used to estimate cum
	 63 	 ls of   implementation of the tMLE est

MAT-Cervical Disease-2019-Greene-LEEP.pdf 296
TB-TB-2018-Churchyard-Bedaquiline-Delamanid.pdf 119
MAL-Malaria-2019-Foy-RIMDAMAL.pdf 58
TB-TB-2019-N-S.pdf 224
NUT-Stunting-2019-Humphrey-SHINE.pdf 91
PNE-Pneumonia-2018-Keenan-Mortality Reduction After Oral Azithromycin.pdf 184
EDD-Cholera-2016-Qadri-Shanchol.pdf 124
TB-TB-2020-John-Stewart-iTIPS.pdf 85
VAC-Ecoli-2019-Qadri-ETVAX.pdf 65
HIV-HIV-2018-Havlir-SEARCH.pdf 101
HIV-HIV-2016-Kurth-High-yield HIV testing, facilitated linkage to care.pdf 111
HIV-HIV-2017-Corey-HVTN704-HPTN085.pdf 175
	 60 	 ximum  likelihood estimation (tMLE) will be used to estimate cum
	 60 	 ails of implementation of the tMLE  estimators in an automated a
	 60 	  (mAb group/control) of these tMLE estimators. Point estimates a
	 121 	 FV tenofovir  TG transgender  tMLE targeted maximum likelihood e
HIV-HIV-2016_Tebas-A5354.pdf 133
HIV-HIV-2016-Weinberg-NICHP1091.pdf 93
MAL-Malaria-2016-Valencia-Phase 1 and 2a Clinical.pdf 61
NTD-Filariasis-2019-Weil-DOLF IDA Indo

In [32]:
len(files)

77